# Create database

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project="artefact-taxonomy")

In [ ]:
sql = """
WITH
 count_sample AS (
 SELECT
   mirakl_model_code,
   COUNT(*) AS nb_sample
 FROM
   `artefact-taxonomy.pdp_product_entity_matching.master_attribute_extraction_training_data`
 WHERE
   attribute_code IN ( "02419",
     "15344",
     "01746",
     "00562")
 GROUP BY
   mirakl_model_code
 HAVING
   COUNT(*)>500
 LIMIT
   100 )
SELECT
 training_data.id,
 training_data.adeo_product_id,
 training_data.title,
 training_data.description,
 training_data.lang,
 training_data.attribute_code,
 training_data.attribute_label,
 training_data.attribute_type,
 training_data.is_multivalued,
 training_data.is_lov,
 training_data.attribute_value,
 training_data.lov_code,
 training_data.step_model_code,
 training_data.mirakl_model_code,
 training_data.data_source
FROM
 `artefact-taxonomy.pdp_product_entity_matching.master_attribute_extraction_training_data` AS training_data
INNER JOIN
 count_sample cs
ON
 cs.mirakl_model_code=training_data.mirakl_model_code
WHERE
 attribute_code IN ( "02419",
   "15344",
   "01746",
   "00562")
"""

df = client.query(sql).to_dataframe()

# Save query results

In [ ]:
table_id = "artefact-taxonomy.pem_uc_add_datasets.temp_simplon_2"
job_config = bigquery.QueryJobConfig(destination=table_id)

In [ ]:
# Start the query, passing in the extra configuration.
query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_id))